# Entrenamiento del Modelo de Detección de Ciclistas

Este notebook cubre el proceso completo para entrenar un modelo de CNN para detectar bicicletas en fotogramas de video. El proceso incluye:

1.  **Extracción de Fotogramas:** Extraer fotogramas de videos de muestra.
2.  **Generación de Archivo de Etiquetas:** Crear un `labels.csv` para que el usuario lo complete.
3.  **Carga y Preprocesamiento de Datos:** Cargar los fotogramas y etiquetas, y prepararlos para el entrenamiento.
4.  **Construcción y Entrenamiento del Modelo:** Definir y entrenar un modelo de CNN.
5.  **Guardado del Modelo:** Guardar el modelo entrenado para su uso en la aplicación.

## 1. Importar Librerías

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import shutil

## 2. Preparación de Datos

### 2.1. Preparación de Datos de Entrenamiento (Automatizado)

Esta celda automatiza la preparación de datos. Si encuentra videos en `../data`, extraerá fotogramas. Si no, creará imágenes de demostración para asegurar que el notebook se pueda ejecutar de principio a fin sin intervención manual.

In [ ]:
def setup_training_data(video_dir='../data', frames_dir='../data/frames', labels_csv='../data/labels.csv'):
    # 1. Preparar el directorio de fotogramas
    if not os.path.exists(frames_dir):
        os.makedirs(frames_dir)
    else:
        shutil.rmtree(frames_dir)
        os.makedirs(frames_dir)
    
    # 2. Extraer fotogramas si existen videos
    video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mp4', '.avi'))]
    frame_count = 0
    if video_files:
        print(f"Videos encontrados: {video_files}. Extrayendo fotogramas...")
        for video_file in video_files:
            video_path = os.path.join(video_dir, video_file)
            cap = cv2.VideoCapture(video_path)
            fps = cap.get(cv2.CAP_PROP_FPS)
            if fps == 0: continue
            frame_interval = int(fps) # Un fotograma por segundo
            while cap.isOpened():
                frame_id = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                ret, frame = cap.read()
                if not ret: break
                if frame_id % frame_interval == 0:
                    cv2.imwrite(os.path.join(frames_dir, f"frame_{frame_count:04d}.jpg"), frame)
                    frame_count += 1
            cap.release()
        print(f"{frame_count} fotogramas extraídos.")
    else:
        print("No se encontraron videos. Creando 100 imágenes de demostración...")
        for i in range(100):
            dummy_image = np.zeros((100, 100, 3), dtype=np.uint8)
            cv2.imwrite(os.path.join(frames_dir, f"frame_{i:04d}.jpg"), dummy_image)
        frame_count = 100

    # 3. Generar y simular el archivo de etiquetas
    frame_files = sorted([f for f in os.listdir(frames_dir) if f.endswith('.jpg')])
    if not frame_files:
        raise ValueError("No se generaron fotogramas. No se puede continuar.")
    
    df = pd.DataFrame({'frame': frame_files})
    # Simular etiquetado aleatorio para que el split funcione
    df['has_bicycle'] = np.random.randint(0, 2, df.shape[0])
    df.to_csv(labels_csv, index=False)
    print(f"Archivo '{labels_csv}' generado y simulado con {len(df)} etiquetas aleatorias.")

# Ejecutar la preparación de datos automatizada
setup_training_data()

## 3. Cargar y Preprocesar Datos

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32

labels_df = pd.read_csv('../data/labels.csv')
labels_df['has_bicycle'] = labels_df['has_bicycle'].astype(str)

train_df, val_df = train_test_split(labels_df, test_size=0.2, random_state=42, stratify=labels_df['has_bicycle'])

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='../data/frames',
    x_col='frame',
    y_col='has_bicycle',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary',
    batch_size=BATCH_SIZE
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='../data/frames',
    x_col='frame',
    y_col='has_bicycle',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary',
    batch_size=BATCH_SIZE
)

## 4. Construcción y Entrenamiento del Modelo

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Entrenar el modelo (si hay datos)
if train_generator.n > 0 and validation_generator.n > 0:
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.n // BATCH_SIZE,
        epochs=10, # En un caso real, usar más épocas
        validation_data=validation_generator,
        validation_steps=validation_generator.n // BATCH_SIZE
    )
else:
    print("No hay suficientes datos para entrenar. Por favor, verifica tus datos y etiquetas.")

## 5. Guardar el Modelo Entrenado

In [ ]:
model.save('../bicycle_detection_model.h5')
print("Modelo guardado como '../bicycle_detection_model.h5'")